###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Instituto Nacional de Salud

In [2]:
import pandas as pd

###Archivo nde2-wquc.csv

Contiene la información sobre el Índice de riesgo de la calidad del agua (IRCA) por municipio del Pais

In [3]:
df = pd.read_csv("https://www.datos.gov.co/resource/nde2-wquc.csv?$limit=500000", dtype={'codigo_mun': str})
df.head()

,a_o,codigo_dep,departamento,codigo_mun,municipio,no_muestras,irca,nivel_de_riesgo
0,2018,91,Amazonas,91001,Leticia,95,19.85,Medio
1,2018,91,Amazonas,91540,Puerto Nariño,24,28.88,Medio
2,2018,5,Antioquia,5002,Abejorral,67,0.69,Sin riesgo
3,2018,5,Antioquia,5004,Abriaquí,35,4.36,Sin riesgo
4,2018,5,Antioquia,5021,Alejandría,36,2.50,Sin riesgo


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   a_o              1050 non-null   int64  
 1   codigo_dep       1050 non-null   int64  
 2   departamento     1050 non-null   object 
 3   codigo_mun       1050 non-null   object 
 4   municipio        1050 non-null   object 
 5   no_muestras      1050 non-null   int64  
 6   irca             1050 non-null   float64
 7   nivel_de_riesgo  1050 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 65.8+ KB


### Eliminación de columnas irrelevantes para el proyecto

In [6]:
relevant_cols = ['a_o', 'departamento', 'codigo_mun', 'municipio', 'no_muestras', 'irca', 'nivel_de_riesgo']
water_quality_risk_Insalud = df[relevant_cols]
water_quality_risk_Insalud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   a_o              1050 non-null   int64  
 1   departamento     1050 non-null   object 
 2   codigo_mun       1050 non-null   object 
 3   municipio        1050 non-null   object 
 4   no_muestras      1050 non-null   int64  
 5   irca             1050 non-null   float64
 6   nivel_de_riesgo  1050 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 57.5+ KB


- Periodo de los datos

In [7]:
water_quality_risk_Insalud['a_o'].unique()

array([2018])

### Verificar valores nulos

In [8]:
water_quality_risk_Insalud.isnull().sum()

,0
a_o,0
departamento,0
codigo_mun,0
municipio,0
no_muestras,0
irca,0
nivel_de_riesgo,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [13]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'codigo_mun', 'municipio', 'nivel_de_riesgo']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(water_quality_risk_Insalud[column].unique())
    print()


Categorías en la columna 'departamento':
['Amazonas' 'Antioquia' 'Arauca'
 'Archipiélago de San Andrés, Providencia y Santa Catalina' 'Atlántico'
 'Bogotá, D.C.' 'Bolívar' 'Boyacá' 'Caldas' 'Caquetá' 'Casanare' 'Cauca'
 'Cesar' 'Chocó' 'Córdoba' 'Cundinamarca' 'Guainía' 'Guaviare' 'Huila'
 'La Guajira' 'Magdalena' 'Meta' 'Nariño' 'Norte de Santander' 'Putumayo'
 'Quindio' 'Risaralda' 'Santander' 'Sucre' 'Tolima' 'Valle del Cauca'
 'Vaupés' 'Vichada']

Categorías en la columna 'codigo_mun':
['91001' '91540' '5002' ... '99524' '99001' '99624']

Categorías en la columna 'municipio':
['Leticia' 'Puerto Nariño' 'Abejorral' 'Abriaquí' 'Alejandría' 'Amagá'
 'Amalfi' 'Andes' 'Angelópolis' 'Angostura' 'Anorí' 'Anza' 'Apartadó'
 'Arboletes' 'Argelia' 'Armenia' 'Barbosa' 'Bello' 'Belmira' 'Betania'
 'Betulia' 'Briceño' 'Buriticá' 'Cáceres' 'Caicedo' 'Caldas' 'Campamento'
 'Cañasgordas' 'Caracolí' 'Caramanta' 'Carepa' 'Carolina' 'Caucasia'
 'Chigorodó' 'Cisneros' 'Ciudad Bolívar' 'Cocorná' 'Concep

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [14]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [15]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    water_quality_risk_Insalud.loc[:,col] = water_quality_risk_Insalud[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "codigo_mun" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [17]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_mun" en el df water_quality_risk_Insalud

In [18]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
water_quality_risk_Insalud.loc[:,'codigo_mun'] = water_quality_risk_Insalud['codigo_mun'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = water_quality_risk_Insalud['codigo_mun'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [19]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5 4]


In [20]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,codigo_mun,count,percentage
0,5,904,0.860952
1,4,146,0.139048


In [21]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = water_quality_risk_Insalud[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_mun']])
    print()

Muestra de registros con longitud 5:
    codigo_mun
0        91001
1        91540
125      81001
126      81065
127      81220

Muestra de registros con longitud 4:
  codigo_mun
2       5002
3       5004
4       5021
5       5030
6       5031



In [22]:
# Verificar longitudes de 'cod_mupi' igual a 3 y 7 dígitos
cod_muni_3_6_7 = water_quality_risk_Insalud[
    (water_quality_risk_Insalud['codigo_mun'].apply(len) == 4)
]
cod_muni_3_6_7['departamento'].value_counts()

,count
departamento,
ANTIOQUIA,123
ATLANTICO,23


Nota: Como se aprecia los 'codigo_mun' con longitudes de 4 dígitos, corresponden a los departamentos de ANTIOQUIA Y ATLANTICO. Estos departamentos tienen codigo que 05 y 08 respectivamente, al parecer en alguna etapa de su generacion el 0 inicial fue suprimido.

 - Adicionar un cero a los codigo_dane de 4 dígitos

In [23]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_4(codigo):
    if len(codigo) == 4:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
water_quality_risk_Insalud.loc[:,'codigo_mun'] = water_quality_risk_Insalud['codigo_mun'].apply(add_zero_if_length_4)

In [24]:
# Verificar cuantos codigo_dane de 4 dígitos quedaron
len(water_quality_risk_Insalud[water_quality_risk_Insalud['codigo_mun'].str.len() == 4])

0

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [25]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [26]:
# Comparar listas de códigos
compare_lists(water_quality_risk_Insalud['codigo_mun'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en water_quality_risk_Insalud", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en water_quality_risk_Insalud que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [29]:
# Eliminar columnas que ya no se necesitan
columns_to_drop = ['departamento', 'municipio']
water_quality_risk_Insalud = water_quality_risk_Insalud.drop(columns=columns_to_drop)

In [30]:
# Adicionar columna para trazabilidad de la fuente
water_quality_risk_Insalud['source_id'] = 122

In [31]:
water_quality_risk_Insalud.columns

Index(['a_o', 'codigo_mun', 'no_muestras', 'irca', 'nivel_de_riesgo',
       'source_id'],
      dtype='object')

In [34]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'a_o': 'year',
    'codigo_mun': 'dane_code',
    'no_muestras': 'number_of_samples',
    'irca': 'irca',
    'nivel_de_riesgo': 'risk_level',
    'source_id': 'source_id'
}

# Renombrar las columnas
water_quality_risk_Insalud.rename(columns=translation_map, inplace=True)

In [35]:
#Estructura final del dataset a integrar a la base de datos
water_quality_risk_Insalud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               1050 non-null   int64  
 1   dane_code          1050 non-null   object 
 2   number_of_samples  1050 non-null   int64  
 3   irca               1050 non-null   float64
 4   risk_level         1050 non-null   object 
 5   source_id          1050 non-null   int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 49.3+ KB


## Salvar en archivo csv en el drive

In [36]:
# Guardar en archivos CSV en el drive
water_quality_risk_Insalud.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/water_quality_risk_Insalud.csv', index=False)